# Assignment 12

### 1.复习上课内容

### 2.回答以下理论问题

#### 1. 请写一下TF-IDF的计算公式
𝑇𝐹−𝐼𝐷𝐹(𝑥)=𝑇𝐹(𝑥)∗𝐼𝐷𝐹(𝑥)<br>

𝑇𝐹(𝑥)指词𝑥在当前文本(句子)中的词频。<br>
𝐼𝐷𝐹(𝑥)=𝑙𝑜𝑔(𝑁/𝑁(𝑥))  N表示总的文本（句子）的个数，𝑁(𝑥) 表示含有单词x的文件（句子）的个数。<br>
但是有时候一些生僻词在语料库中没有，就需要对𝐼𝐷𝐹(𝑥)做一些平滑，以免防止分母为零，没有意义。一种平滑的方法为： 𝐼𝐷𝐹(𝑥)=𝑙𝑜𝑔(𝑁+1/𝑁(𝑥)+1)   +1

#### 2. LDA算法的基本假设是什么？
第一个假设： Per-document topic distributions: The topics of each document obey a probability distribution.
文本的主题会符合某一个概率分布。 比如新闻0.3， 经济0.2， 娱乐0.3 等。<br>
第二个假设: Per-topic word distributions: The words of each topic obey  a probability distribution. 
每一个主题下的词符合某一个概率分布。 比如新闻主题里面国家，外交这些词就会比较多。

#### 3. 在TextRank算法中构建图的权重是如何得到的？
TextRank用一个长度为N的滑动窗口，认为所有在这个窗口之内的词都有一条无向的边连接。 这条边的权重由两个词的词向量之间的similarity（余旋相似度）来决定。

#### 4. 什么是命名实体识别？ 有什么应用场景？
Named entity recognition(NER) 是指识别文本中具有特定意义的实体，主要包括人名、地名、机构名、专有名词，并且标注其词的属性。
命名实体识别的任务就是识别出待处理文本中三大类（实体类、时间类和数字类）、七小类（人名、机构名、地名、时间、日期、货币和百分比）。

NER可以用于信息提取、问答系统、句法分析、机器翻译、元数据标注等应用领域的重要基础工具。



#### 5.NLP主要有哪几类任务 ？
第一大类是：TEXT Classification 文本分类。一般采用的方法是先做文本表示，用一个向量来表示文本，常见方法有:TF-IDF， Word-Vector（gensim model), 词向量加模型（RNN，LSTM）。然后加上分类器（传统机器学习方法:Naive Bayes，SVM等； 或者用deep learning 神经网络（RNN LSTM，Transformer，GRU）。常见的文本分类Sentiment Analysis 是seqto1 的文本分类任务，输入一个句子只需要输出一个结果（postive，negative 或者neutral）。 
也有一些分类任务是seq2seq的，比如NER或者POS，输入一个句子要对每一个词都做词性标注，这就相当于输入一个seq作为input要输出一个seq为output，相当于同时要做几个分类器。

第二大类是：文本生成。
包括机器翻译，常用方法为：RNN， RNN+Attention， Transformer


其他问题： 问答系统，通过一个输入的文本来回答问题。 或者通过输入文本+图片+语音来回答问题》

### 3.实践题

#### 3.1 手动实现TextRank算法 (在新闻数据中随机提取100条新闻训练词向量和做做法测试）

 提示：
 1. 确定窗口，建立图链接。   
 2. 通过词向量相似度确定图上边的权重
 3. 根据公式实现算法迭代(d=0.85)

In [1]:
import pandas as pd
import re
import random
path='/Users/zixiawang/school/kaikeba/作业笔记/开课吧NLP/Project/Project1/Unsupervise_news_abstract/raw_data'
df=pd.read_csv(path+'/sqlResult_1558435.csv',encoding='GB18030')
print (len(df))
df=df[~df['content'].isnull()]
print (len(df))
df=df.sample(n = 100,random_state=1) 
print (len(df))

89611
87054
100


In [2]:
sequence=' '.join(list(df.content)).replace('\r\n', '').replace('\u3000', '').replace('\\n', '')
sequence=' '.join(re.findall('\w+',sequence))
sequence=' '.join(re.findall('\D+',sequence))
print (sequence[:100])

import jieba
seg_list=list(jieba.cut(sequence, cut_all=False))
print (len(seg_list))
seg_list= [x for x in seg_list if x != ' ']
print (len(seg_list))

新华社安卡拉 月 日电据土耳其ＮＴＶ电视台报道 土东南部城市迪亚巴克尔 日上午发生爆炸 造成至少 人受伤 爆炸发生在位于迪亚巴克尔市内省安全局大楼附近 事发时 一辆警用装甲车正在事发地修理 目前尚无任


Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/cc/rpzkx16j4bg9cwh9pmkgzs2w0000gn/T/jieba.cache
Loading model cost 0.580 seconds.
Prefix dict has been built succesfully.


27312
21032


### Train word2vec model to get word vector and similarity between words using all the news data

In [272]:
##generate segment files
result = ' '.join(seg_list)
result = result.encode('utf-8')
with open('./segment/segment.txt', 'wb') as f2:
    f2.write(result)

from gensim.models import word2vec
import multiprocessing
# 如果目录中有多个文件，可以使用PathLineSentences
sentences = word2vec.PathLineSentences('./segment')
# 设置模型参数，进行训练
import time
start_time = time.time()
model2 = word2vec.Word2Vec(sentences, size=100, window=5, min_count=1,workers=multiprocessing.cpu_count())
#print(model2.wv.similarity('北京', '时间'))
#print(model2.wv.similarity('爸爸', '妈妈'))
#save the model
#!ls ../Project/models
model2.save('../Project/models/news_model2.model')
print ("My program took", time.time() - start_time, "to run")

My program took 100.51415205001831 to run


In [275]:
#!ls ../Project/models
from gensim.models import word2vec
model = word2vec.Word2Vec.load("../Project/models/news_model2.model")
print (len(model2.wv.vocab.items()))
print (list(model2.wv.vocab.keys())[:20])

240453
['此外', '自', '本周', '月', '日', '起', '除', '小米', '手机', '等', '款', '机型', '外', '其余', '已', '暂停', '更新', '发布', '含', '开发']


In [276]:
print (model2.wv.most_similar('新华社'))

[('新华社摄影部', 0.5894807577133179), ('综摄', 0.5647972822189331), ('张纹', 0.5542241334915161), ('李挺摄', 0.5513587594032288), ('中新社', 0.5410710573196411), ('达伍', 0.5288800001144409), ('王正摄', 0.5208887457847595), ('尼摄', 0.5203403234481812), ('缗', 0.5179715156555176), ('珂', 0.5119003057479858)]


In [277]:
word_vectors = model2.wv
print (word_vectors['新华社'].shape)
print(word_vectors.similarity('新华社', '电视台'))

(100,)
0.02208627


In [278]:
print(word_vectors.similarity('武力', '和平'))

0.5141018


In [279]:
print(word_vectors.similarity('罪犯', '好人'))

0.08937922


In [212]:
?word_vectors.similarity()

In [ ]:
v1=word_vectors['足球']
v2=word_vectors['欧冠']

word list in the 100 news

In [187]:
!ls ../Project/stopwords

README.md
中文停用词表.txt
百度停用词表.txt
中文停用词表_small.txt
哈工大停用词表.txt
四川大学机器智能实验室停用词库.txt


In [230]:
import codecs
stopwords = codecs.open('../Project/stopwords/中文停用词表_small.txt', 'r', 'utf-8').read().replace('\n',',').split(',')
print (len(stopwords))

738


In [284]:
print (len(seg_list))
seg_list= [x for x in seg_list if x not in stopwords]
print (len(seg_list))
seg_list

21032
16976


['新华社',
 '安卡拉',
 '月',
 '日电',
 '土耳其',
 'Ｎ',
 'Ｔ',
 'Ｖ',
 '电视台',
 '报道',
 '土',
 '东南部',
 '城市',
 '迪亚巴克尔',
 '日',
 '上午',
 '发生爆炸',
 '造成',
 '至少',
 '受伤',
 '爆炸',
 '发生',
 '位于',
 '迪亚巴克尔',
 '市内',
 '省',
 '安全局',
 '大楼',
 '附近',
 '事发',
 '时',
 '一辆',
 '警用',
 '装甲车',
 '正在',
 '事发',
 '修理',
 '目前',
 '尚无',
 '组织',
 '个人',
 '宣称',
 '制造',
 '爆炸',
 '完',
 '新华社',
 '上海',
 '月',
 '日电',
 '记者',
 '王琳琳',
 '北斗',
 '与物',
 '联网',
 '数据',
 '新',
 '技术',
 '深度',
 '融合',
 '不断',
 '催生出',
 '新',
 '应用',
 '业态',
 '正在',
 '举行',
 '第八届',
 '中国',
 '卫星',
 '导航',
 '学术年会',
 '中国',
 '卫星',
 '导航系统',
 '管理',
 '办公室',
 '主任',
 '冉承其',
 '介绍',
 '中国',
 '北斗',
 '卫星',
 '导航',
 '定位系统',
 '多个',
 '共享',
 '单车',
 '玩家',
 '合作',
 '利用',
 '北斗',
 '地基',
 '增强',
 '系统',
 '实现',
 '米级',
 '亚米级',
 '高',
 '精准',
 '时空',
 '定位',
 '单车',
 '单车',
 '究竟',
 '骑过',
 '共享',
 '单车',
 '可能',
 '遭遇',
 '手机',
 'APP',
 '看到',
 '不远处',
 '一辆',
 '单车',
 '走',
 '地图',
 '标出',
 '位置',
 '点',
 '一辆',
 '单车',
 '没有',
 '红包',
 '单车',
 '明明',
 '显示',
 '附近',
 '找',
 '半天',
 '根本',
 '知道',
 '具体',
 '一辆',
 '出现',
 '定位',
 '漂移',
 '根本原因',
 '导航系统',
 '时空',
 '

#### prepare (word1, word2, similarity) as graph data for textrank calculation
#### only support window_size<=6

In [285]:
graph_data=[]
def get_graph_data(window_size):
    for i in range(len(seg_list)-6):
        #print (i)
        one_window_record=(seg_list[i], seg_list[i+1], np.absolute(model2.wv.similarity(seg_list[i], seg_list[i+1])))
        two_window_record=(seg_list[i], seg_list[i+2],  np.absolute(model2.wv.similarity(seg_list[i], seg_list[i+2])))
        three_window_record=(seg_list[i], seg_list[i+3], np.absolute(model2.wv.similarity(seg_list[i], seg_list[i+3])))
        four_window_record=(seg_list[i], seg_list[i+4],  np.absolute(model2.wv.similarity(seg_list[i], seg_list[i+4])))
        five_window_record=(seg_list[i], seg_list[i+5],  np.absolute(model2.wv.similarity(seg_list[i], seg_list[i+5])))
        six_window_record=(seg_list[i], seg_list[i+6],  np.absolute(model2.wv.similarity(seg_list[i], seg_list[i+6])))
        if window_size==1:
            graph_data.append(one_window_record)
        elif window_size==2:
            graph_data.append(one_window_record)
            graph_data.append(two_window_record)
        elif window_size==3:
            graph_data.append(one_window_record)
            graph_data.append(two_window_record)
            graph_data.append(three_window_record)
        elif window_size==4:
            graph_data.append(one_window_record)
            graph_data.append(two_window_record)
            graph_data.append(three_window_record)
            graph_data.append(four_window_record)
        elif window_size==5:
            graph_data.append(one_window_record)
            graph_data.append(two_window_record)
            graph_data.append(three_window_record)
            graph_data.append(four_window_record)
            graph_data.append(five_window_record)
        else :
            graph_data.append(one_window_record)
            graph_data.append(two_window_record)
            graph_data.append(three_window_record)
            graph_data.append(four_window_record)
            graph_data.append(five_window_record)
            graph_data.append(six_window_record)
        i=i+1
    return graph_data
graph_data=get_graph_data(6)  

In [286]:
len(graph_data)

101820

In [287]:
graph_data[:50]

[('新华社', '安卡拉', 0.3046392),
 ('新华社', '月', 0.404448),
 ('新华社', '日电', 0.25547475),
 ('新华社', '土耳其', 0.11192436),
 ('新华社', 'Ｎ', 0.043917745),
 ('新华社', 'Ｔ', 0.14521271),
 ('安卡拉', '月', 0.26072228),
 ('安卡拉', '日电', 0.12852746),
 ('安卡拉', '土耳其', 0.55630475),
 ('安卡拉', 'Ｎ', 0.026355283),
 ('安卡拉', 'Ｔ', 0.09328106),
 ('安卡拉', 'Ｖ', 0.15385437),
 ('月', '日电', 0.053360775),
 ('月', '土耳其', 0.10911085),
 ('月', 'Ｎ', 0.008820509),
 ('月', 'Ｔ', 0.17707217),
 ('月', 'Ｖ', 0.15443918),
 ('月', '电视台', 0.11195049),
 ('日电', '土耳其', 0.0014949942),
 ('日电', 'Ｎ', 0.13895306),
 ('日电', 'Ｔ', 0.06036166),
 ('日电', 'Ｖ', 0.023176502),
 ('日电', '电视台', 0.14791672),
 ('日电', '报道', 0.26552728),
 ('土耳其', 'Ｎ', 0.15091328),
 ('土耳其', 'Ｔ', 0.13716295),
 ('土耳其', 'Ｖ', 0.020167746),
 ('土耳其', '电视台', 0.21539529),
 ('土耳其', '报道', 0.2886632),
 ('土耳其', '土', 0.26419643),
 ('Ｎ', 'Ｔ', 0.74650884),
 ('Ｎ', 'Ｖ', 0.7153649),
 ('Ｎ', '电视台', 0.21918882),
 ('Ｎ', '报道', 0.19889276),
 ('Ｎ', '土', 0.008323822),
 ('Ｎ', '东南部', 0.022528447),
 ('Ｔ', 'Ｖ', 0.8683253),
 ('

In [288]:
import networkx as nx
def get_textrank(graph_data):
    graph = nx.DiGraph()
    graph.add_weighted_edges_from(graph_data)
    pagerank = nx.pagerank(graph,max_iter=100)
    pagerank=pd.Series(pagerank).to_frame('pagerank').reset_index().rename(columns={'index':'word'})
    pagerank=pagerank.sort_values(by='pagerank',ascending=False)
    return pagerank

pagerank=get_textrank(graph_data)

### those are the top 20 keywords from the top 100 news based on pagerank using cosine similarity as weight

In [289]:
pagerank.head(20)

,word,pagerank
59,中国,0.004992
0,新华社,0.004257
2,月,0.003915
14,日,0.003492
1311,工作,0.003456
4902,机关,0.003279
772,科技,0.003223
568,创新,0.002939
247,年,0.002933
273,发展,0.002618


### compare result with textrank4zh method

In [426]:
df=df.sample(n = 100,random_state=1) 
sequence=' '.join(list(df.content)).replace('\r\n', '').replace('\u3000', '').replace('\\n', '')
sequence=' '.join(re.findall('\w+',sequence))
sequence=' '.join(re.findall('\D+',sequence))
print (sequence[:100])

新华社照片 杜塞尔多夫  年 月 日 体育   乒乓球 世乒赛 马龙 波尔晋级男双 强 月 日 马龙 前 波尔在比赛中 当日 在德国杜塞尔多夫举行的 年世界乒乓球锦标赛男双第一轮比赛中 马龙 中国 波


In [424]:
from textrank4zh import TextRank4Keyword, TextRank4Sentence
text=sequence
# 输出关键词，设置文本小写，窗口为2
tr4w = TextRank4Keyword()
tr4w.analyze(text=text, lower=True, window=2)
print('关键词：')
for item in tr4w.get_keywords(20, word_min_len=1):
    print(item.word, item.weight)

关键词：
中国 0.008277774977006717
日 0.0067698170520256165
新华社 0.0044467473459943625
上 0.004152478589225504
记者 0.003841655174739745
工作 0.0037772985532475256
科技 0.0037633625726840466
发展 0.0034661937647641116
博物馆 0.0030430448625334977
不 0.0028230723877360918
国家 0.002739984585418673
创新 0.0027102221053222643
说 0.0025721315731607
国际 0.002307015658199976
世界 0.0022640628109316815
文化 0.00224917356457295
新 0.0021874661941361797
建设 0.0021820817811147105
进行 0.0021259049758497405
社会 0.0020999000817314953


In [428]:
# 输出重要的句子
#tr4s = TextRank4Sentence()
#tr4s.analyze(text=text, lower=True, source = 'all_filters')
#print('摘要：')
# 重要性较高的三个句子
#for item in tr4s.get_key_sentences(num=3):
	# index是语句在文本中位置，weight表示权重
    #print(item.index, item.weight, item.sentence)

### 3.2 使用词向量和k-means的方法寻找关键词

提示：   
1.使用3.1训练好的词向量   
2.可使用sklearn等机器学习库

思路：
Train a model to learn word vector. <br>
Use k-means algorithm to cluster words.<br>
Compute the distance of each word to the center of the cluster it belongs to.<br>
Pick the nearest K words as key words.<br>

In [296]:
import re
from collections import Counter
## clean up the segement text before count the frequency

def get_chinese_number_letters(train):
    #train = train_file
    #train = open(train,encoding="utf8", errors='ignore').read().lower()
    #[\u00C0-\u017F] Unicode range for all Latin characters
    #[\u4e00-\u9fff] Unicode range for all chinese characters
    
    chinese_only=re.findall(r'([\u4e00-\u9fff]+|[A-Z]+|[0-9]+|[a-z]+| [ ])',train)
    
    #chinese_only=re.findall('[\w\s]',train)
    print ('First 20 characters of the train file is :', ' '.join(chinese_only)[:20])
    return ' '.join(chinese_only)

def get_1_gram_count(text):
    print ('Length of cleaned train file: ', len(text))
    TOKENS = text.split(' ')
    print ('Train file contains ', len(TOKENS), ' of words(tokens).')
    _1_gram_word_counts = Counter(TOKENS)
    return _1_gram_word_counts


start_time = time.time()
sequence=' '.join(seg_list)
clean_file=get_chinese_number_letters(sequence)
_1_gram_word_counts=get_1_gram_count(clean_file)
print ("My program took", time.time() - start_time, "to run")

First 20 characters of the train file is : 新华社 安卡拉 月 日电 土耳其 电视台
Length of cleaned train file:  53330
Train file contains  16966  of words(tokens).
My program took 0.013988018035888672 to run


In [300]:
import pandas as pd
wordfreq=dict(_1_gram_word_counts)
word_count_df=pd.DataFrame(list(wordfreq.items()),columns=['word', 'freq'])
word_count_df.head(10)

,word,freq
0,新华社,148
1,安卡拉,1
2,月,186
3,日电,21
4,土耳其,1
5,电视台,1
6,报道,17
7,土,1
8,东南部,1
9,城市,17


In [301]:
word_count_df.describe(percentiles=[0.05,0.8,0.9, 0.95])

,freq
count,5741.000000
mean,2.955234
std,8.991280
min,1.000000
5%,1.000000
50%,1.000000
80%,3.000000
90%,5.000000
95%,9.000000
max,291.000000


In [384]:
sub_words_df=word_count_df[word_count_df.freq>=2].word.to_frame()
sub_words_df.head()

,word
0,新华社
2,月
3,日电
6,报道
9,城市


In [385]:
sub_words_df['word_vector']=sub_words_df.word.apply(lambda x: word_vectors[x])

In [386]:
sub_words_df.head()

,word,word_vector
0,新华社,"[-1.2394855, 0.36834607, 1.146711, -3.191303, ..."
2,月,"[-3.965865, -0.06224927, 4.3941665, -0.1249184..."
3,日电,"[0.07256763, -1.3682668, 1.5509152, -2.4947715..."
6,报道,"[-0.8730091, -0.7818882, 0.2703968, 2.0662866,..."
9,城市,"[3.4460964, 2.0520113, 2.295242, 1.0522008, 1...."


In [387]:
vector_values=[]
for i in sub_words_df.word_vector:
    for value in i:
        vector_values.append(value)
len(vector_values)

234000

In [388]:
matrix=np.reshape(vector_values, (2340, 100))
matrix.shape

(2340, 100)

In [389]:
type(matrix)

numpy.ndarray

### Based on gridsearch, I decide to use n =3 as the number of cluster

In [390]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
range_n_clusters = [2,3,4,5,6,7,8,9,10]
for n_clusters in range_n_clusters:
    clusterer = KMeans(n_clusters=n_clusters)
    cluster_labels = clusterer.fit_predict(matrix)
    silhouette_avg = silhouette_score(matrix, cluster_labels)
    print("For n_clusters =", n_clusters, "The average silhouette_score is :", silhouette_avg)

For n_clusters = 2 The average silhouette_score is : 0.09714786
For n_clusters = 3 The average silhouette_score is : 0.102401406
For n_clusters = 4 The average silhouette_score is : 0.08222686
For n_clusters = 5 The average silhouette_score is : 0.067947306
For n_clusters = 6 The average silhouette_score is : 0.063454196
For n_clusters = 7 The average silhouette_score is : 0.05894347
For n_clusters = 8 The average silhouette_score is : 0.044235405
For n_clusters = 9 The average silhouette_score is : 0.04321716
For n_clusters = 10 The average silhouette_score is : 0.052708052


In [392]:
km = KMeans(n_clusters=3)
sub_words_df['cluster'] = km.fit_predict(matrix)
sub_words_df.cluster.value_counts()

1    1345
2     817
0     178
Name: cluster, dtype: int64

In [400]:
alldistance=km.fit_transform(matrix)
alldistance

array([[21.02583  , 19.119366 , 18.414415 ],
       [22.310677 , 20.12844  , 19.805466 ],
       [42.343925 , 40.76915  , 40.314037 ],
       ...,
       [ 6.1929913,  2.1095104,  8.431876 ],
       [15.985444 , 16.062838 , 20.479076 ],
       [16.747023 , 14.186081 , 15.6935005]], dtype=float32)

In [401]:
mindistances = np.min(km.fit_transform(matrix),axis=1)
print (len(mindistances))
mindistances

2340


array([18.391523 , 19.790142 , 40.31914  , ...,  2.1008737, 16.024574 ,
       14.160169 ], dtype=float32)

In [402]:
sub_words_df['distance_to_centroid']=mindistances

In [403]:
sub_words_df.head()

,word,word_vector,cluster,distance_to_centroid
0,新华社,"[-1.2394855, 0.36834607, 1.146711, -3.191303, ...",0,18.391523
2,月,"[-3.965865, -0.06224927, 4.3941665, -0.1249184...",0,19.790142
3,日电,"[0.07256763, -1.3682668, 1.5509152, -2.4947715...",0,40.319141
6,报道,"[-0.8730091, -0.7818882, 0.2703968, 2.0662866,...",1,21.681522
9,城市,"[3.4460964, 2.0520113, 2.295242, 1.0522008, 1....",2,20.558979


In [413]:
sub_words_df.groupby(['cluster']).describe(percentiles=[0.01,0.001])

distance_to_centroid                                           \
                       count       mean       std       min      0.1%   
cluster                                                                 
0                      178.0  16.490465  7.083378  5.892505  5.943976   
1                     1345.0  10.081196  6.014675  1.773980  1.881522   
2                      817.0  14.967887  3.138837  5.703432  6.199479   

                                         
               1%        50%        max  
cluster                                  
0        6.196392  15.656381  40.319141  
1        1.974042   9.533073  29.398176  
2        7.470562  15.064001  25.310831

### Top 1% Keywords for cluster 0: 体育类新闻

In [411]:
sub_words_df[sub_words_df.cluster==0][sub_words_df.distance_to_centroid<=6.19]

/Users/zixiawang/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,word,word_vector,cluster,distance_to_centroid
1809,国际米兰,"[-0.2882528, -0.05700181, 0.2027864, 0.0526787...",0,6.183299
4680,朝鲜队,"[-0.5163229, 0.12775166, 1.0922734, 0.36114183...",0,5.892505


### Top 1% Keywords for cluster 1: 各类、杂类新闻

In [415]:
sub_words_df[sub_words_df.cluster==1][sub_words_df.distance_to_centroid<=1.97]

/Users/zixiawang/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,word,word_vector,cluster,distance_to_centroid
249,嵊州市,"[-0.018503278, 0.2600716, 0.1463151, -0.007613...",1,1.890182
640,伯克利分校,"[-0.13582912, -0.020702636, 0.085762896, 0.051...",1,1.961135
1690,中国残疾人福利基金会,"[-0.092851944, 0.12675218, 0.15701583, 0.09844...",1,1.946095
1863,峡乡,"[0.04752054, 0.32943267, 0.28224492, 0.1904741...",1,1.927771
2169,游动,"[-0.018306175, 0.29329762, 0.12811524, 0.13247...",1,1.876981
2176,球菌,"[-0.013607798, 0.25857422, 0.17942224, 0.20228...",1,1.773980
3066,新津县,"[-0.20369293, 0.3077638, 0.16443276, 0.2364800...",1,1.921339
3961,王喜良,"[-0.06148221, 0.12690638, 0.05519384, 0.098186...",1,1.898920
4036,保建彬,"[-0.08810858, 0.08712541, 0.06667748, 0.108388...",1,1.917602
4037,柳文炜,"[-0.09746865, 0.121674806, 0.062238388, 0.1390...",1,1.900257


### Top 1% Keywords for cluster 2: 企业改革类新闻

In [417]:
sub_words_df[sub_words_df.cluster==2][sub_words_df.distance_to_centroid<=7.47]

/Users/zixiawang/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,word,word_vector,cluster,distance_to_centroid
1145,激活,"[1.0493716, 0.27142268, 0.34340197, -0.2467577...",2,6.396455
1658,战斗力,"[0.085501626, -0.6506656, 0.1572162, 0.3621078...",2,6.524720
2311,绩效,"[-1.2042015, -0.50121593, -0.55710214, 0.25419...",2,6.866986
3881,弹性,"[1.1304525, 0.34048674, 0.34739938, 0.2507228,...",2,5.703432
4054,产业结构,"[0.7482879, -0.135199, 1.1493102, -0.32757118,...",2,7.452313
4377,来讲,"[0.71832603, -1.4316448, 0.39904976, 1.0269932...",2,6.680180
5001,效能,"[-0.7753035, -0.48257738, 0.1388276, 0.4649609...",2,6.311333
5386,现状,"[0.9556842, 0.4066693, 1.9429407, 1.2300775, -...",2,7.107759
5625,储备,"[0.36569202, -0.70678824, -0.90667576, 0.09210...",2,7.377763


### 3.3  提取新闻人物里的对话。(使用以上提取小数据即可）

提示：    
1.寻找语料里具有表示说的意思。    
2.使用语法分析提取句子结构。    
3.检测谓语是否有表示说的意思。

In [429]:
model.wv.most_similar('说')

[('表示', 0.8259825110435486),
 ('认为', 0.7800418138504028),
 ('指出', 0.7638667821884155),
 ('告诉', 0.7388713359832764),
 ('坦言', 0.7019120454788208),
 ('看来', 0.6988577842712402),
 ('称', 0.6745396852493286),
 ('中说', 0.6444582939147949),
 ('介绍', 0.6370186805725098),
 ('明说', 0.6177627444267273)]

In [434]:
shuo_list=['说','表示', '认为', '指出', '告诉', '坦言', '看来', '称', '介绍', '明说']
print (shuo_list)

['说', '表示', '认为', '指出', '告诉', '坦言', '看来', '称', '介绍', '明说']


In [3]:
df.content.head()

24558    　　新华社安卡拉４月１１日电据土耳其ＮＴＶ电视台报道，土东南部城市迪亚巴克尔１１日上午发生爆...
73760    \n新华社上海5月24日电（记者王琳琳）北斗与物联网、大数据等新技术的深度融合已不断催生出新...
36341    新华社照片，嵊州（浙江），2017年4月21日\n浙江嵊州：传承越剧焕发光彩\n4月20日，...
69161    新华社照片，石家庄，2017年5月20日\n河北衡水：推广高效节水灌溉拧紧农业“水龙头”\n...
19294    新华社照片，外代，2017年4月7日\n（外代二线）德国埃森举行经典车展\n这是4月5日在德...
Name: content, dtype: object

In [435]:
def filter_shuo(df):
    for i in shuo_list:
        if df.content.find(i)!=-1:
            df['contains_shuo']=0
        else:
            df['contains_shuo']=1
    return df

In [437]:
df=df.apply(filter_shuo, axis=1)
df.contains_shuo.value_counts()

1    99
0     1
Name: contains_shuo, dtype: int64

### sample 的100个句子里面有99个出现了说，或者和说意思类似的词语

参考网页<br>
https://blog.csdn.net/guolindonggld/article/details/72795022 <br>
https://zhuanlan.zhihu.com/p/66495302 <br>
http://ltp.ai/demo.html<br>

### 用jieba 得到postagging

In [40]:
import jieba.posseg
posseg_list=jieba.posseg.cut('我说了一句话')
print ('|'.join('%s/%s' % (word, tag) for (word, tag) in posseg_list))

我/r|说/v|了/ul|一句/m|话/n


In [41]:
def get_postag(df):
    posseg_list=jieba.posseg.cut(df.content)
    df['pos_tags']='|'.join('%s/%s' % (word, tag) for (word, tag) in posseg_list)
    return df

In [42]:
df=df.apply(lambda x: get_postag(x),axis=1)

In [43]:
df.pos_tags.head()

24558    　/x|　/x|新华社/nt|安卡拉/nr|４/x|月/m|１/x|１/x|日电/j|据/p...
73760    \/x|n/x|新华社/nt|上海/ns|5/m|月/m|24/m|日电/j|（/x|记者/...
36341    新华社/nt|照片/n|，/x|嵊州/ns|（/x|浙江/ns|）/x|，/x|2017/m...
69161    新华社/nt|照片/n|，/x|石家庄/nr|，/x|2017/m|年/m|5/m|月/m|...
19294    新华社/nt|照片/n|，/x|外代/n|，/x|2017/m|年/m|4/m|月/m|7/...
Name: pos_tags, dtype: object

In [44]:
df.pos_tags[24558]

'\u3000/x|\u3000/x|新华社/nt|安卡拉/nr|４/x|月/m|１/x|１/x|日电/j|据/p|土耳其/ns|Ｎ/x|Ｔ/x|Ｖ/x|电视台/n|报道/v|，/x|土/zg|东南部/f|城市/ns|迪亚巴克尔/nrt|１/x|１/x|日/m|上午/t|发生爆炸/n|，/x|造成/v|至少/d|１/x|０/x|人/n|受伤/v|。/x|爆炸/v|发生/v|在/p|位于/v|迪亚巴克尔/nrt|市内/s|省/n|安全局/nt|大楼/n|附近/f|。/x|事发/n|时/n|，/x|一辆/m|警用/n|装甲车/n|正在/t|事发/n|地/uv|修理/v|。/x|目前/t|尚无/v|任何/r|组织/v|或/c|个人/n|宣称/v|制造/v|此次/r|爆炸/v|。/x|（/x|完/v|）/x'

In [46]:
df.pos_tags[24558].split('/v|')

['\u3000/x|\u3000/x|新华社/nt|安卡拉/nr|４/x|月/m|１/x|１/x|日电/j|据/p|土耳其/ns|Ｎ/x|Ｔ/x|Ｖ/x|电视台/n|报道',
 '，/x|土/zg|东南部/f|城市/ns|迪亚巴克尔/nrt|１/x|１/x|日/m|上午/t|发生爆炸/n|，/x|造成',
 '至少/d|１/x|０/x|人/n|受伤',
 '。/x|爆炸',
 '发生',
 '在/p|位于',
 '迪亚巴克尔/nrt|市内/s|省/n|安全局/nt|大楼/n|附近/f|。/x|事发/n|时/n|，/x|一辆/m|警用/n|装甲车/n|正在/t|事发/n|地/uv|修理',
 '。/x|目前/t|尚无',
 '任何/r|组织',
 '或/c|个人/n|宣称',
 '制造',
 '此次/r|爆炸',
 '。/x|（/x|完',
 '）/x']

In [ ]:
## 用这个方法run 不出来 

import logging
from stanfordcorenlp import StanfordCoreNLP

nlp = StanfordCoreNLP(r'../Data/stanford-corenlp-full-2016-10-31/', quiet=False, logging_level=logging.DEBUG,lang='zh')
#nlp = StanfordCoreNLP(r'../Data/stanford-corenlp-full-2016-10-31/', lang='zh')

sentence = '清华大学位于北京。'
print (nlp.word_tokenize(sentence))
print (nlp.pos_tag(sentence))
print (nlp.ner(sentence))
print (nlp.parse(sentence))
print (nlp.dependency_parse(sentence))